# Demonstrating the module system in Python and Nipype

## First, let's demonstrate that bet, a tool from the fsl toolkit, is not on our path

In [ ]:
!bet

# Now we can use lmod to load fsl in a specific version

In [ ]:
import lmod
await lmod.load('fsl/6.0.4')
await lmod.list()

In [ ]:
!bet

## Load afni as well

In [ ]:
await lmod.load('afni/22.3.06')
await lmod.list()

## download test data

In [ ]:
![ -f /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii  ] && echo "$FILE exist." || wget https://objectstorage.us-ashburn-1.oraclecloud.com/n/idrvm4tkz2a8/b/TOMCAT/o/TOMCAT_DIB/sub-01/ses-01_7T/anat/sub-01_ses-01_7T_T1w_defaced.nii.gz -O /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz 
![ -f /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii  ] && echo "$FILE exist." || gunzip /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz 

In [ ]:
!ls /home/jovyan/neurodesktop-storage

## run nipype pipeline

In [ ]:
from nipype.interfaces import fsl
from nipype.interfaces import afni

btr = fsl.BET()
btr.inputs.in_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii'
btr.inputs.frac = 0.4
btr.inputs.out_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_brain.nii'
res = btr.run() 

edge3 = afni.Edge3()
edge3.inputs.in_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii'
edge3.inputs.out_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_edges.nii'
edge3.inputs.datum = 'byte'
res = edge3.run()

In [ ]:
!ls /home/jovyan/neurodesktop-storage

In [ ]:
# View 3D data
import matplotlib.pyplot as plt

def view_slices_3d(image_3d, slice_nbr, vmin, vmax, title=''):
#   print('Matrix size: {}'.format(image_3d.shape))
  fig = plt.figure(figsize=(15, 4))
  plt.suptitle(title, fontsize=10)

  plt.subplot(131)
  plt.imshow(np.take(image_3d, slice_nbr, 2), vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Axial');

  plt.subplot(132)
  image_rot = ndimage.rotate(np.take(image_3d, slice_nbr, 1),90)
  plt.imshow(image_rot, vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Coronal');

  plt.subplot(133)
  image_rot = ndimage.rotate(np.take(image_3d, slice_nbr, 0),90)
  plt.imshow(image_rot, vmin=vmin, vmax=vmax, cmap='gray')
  plt.title('Sagittal');
  cbar=plt.colorbar()

def get_figure():
    """
    Returns figure and axis objects to plot on. 
    """
    fig, ax = plt.subplots(1)
    plt.tick_params(top=False, right=False, which='both') 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    return fig, ax

In [ ]:
import nibabel as nib
from matplotlib import transforms
from scipy import ndimage
import numpy as np

# load data
brain_full = nib.load('/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii').get_fdata()
brain = nib.load('/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_brain.nii.gz').get_fdata()
edges = nib.load('/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_edges.nii').get_fdata()

In [ ]:
view_slices_3d(brain_full, slice_nbr=230, vmin=0, vmax=4000, title='Brain and Skull')
view_slices_3d(brain, slice_nbr=230, vmin=0, vmax=4000, title='Brain Extracted')
view_slices_3d(edges, slice_nbr=230, vmin=0, vmax=1000, title='Edges')